# Step 1 - Install the required dependencies and make sure the python version is 3.10 and above

In [ ]:
!pip install zeno-client
!pip install --upgrade pip
!pip install --upgrade bottleneck
!pip install langdetect


In [2]:
!python --version

Python 3.10.13


In [3]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd

# Initialize a client with our API key.
client = ZenoClient("<Enter API key>")


/Users/shauryasingh/Documents/CMU/AI Engineering/cmu-mlip-zeno-lab/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step 2 - Create a project

In [11]:
project = client.create_project(
    name="Biased Movies Analysis",
    view="text-classification",
    metrics=[
        ZenoMetric(name="accuracy", type="mean", columns=["correct"]),
        #Uncomment the following line after implementing the function
        ZenoMetric(name="avg length ratio", type="mean", columns=["avg_length_ratio"]),
    ]
)

Successfully updated project.
Access your project at  https://hub.zenoml.com/project/bacf1e34-5621-4f65-8161-c37df68578ff/Biased%20Movies%20Analysis


# Step 3 - Create dataset

In [12]:
import pandas as pd
from langdetect import detect
from collections import Counter
import random

# Sample language detection function
def detect_language(text):
    try:
        return detect(text)
    except:
        return "Unknown"

# Create a Pandas DataFrame for the biased dataset of movies
df = pd.DataFrame(
    {
        "id": range(1, 11),  # Using only 10 instances for demonstration
        "text": [
            "I love this movie!",
            "¡Odio esta película!",
            "Ce film est ok.",
            "The acting was great!",
            "The plot was confusing.",
            "I really enjoyed it!",
            "Worst movie ever!",
            "Not bad, but could be better.",
            "A masterpiece!",
            "I didn't like it.",
        ],
        "genre": ["action", "drama", "comedy", "action", "drama", "action", "drama", "comedy", "action", "drama"],
        "year": [random.choice([2020, 2021, 2022]) for _ in range(10)],  # Randomly select year
        "label": ["positive", "negative", "neutral", "positive", "negative", "positive", "negative", "neutral", "positive", "negative"],
    }
)

# Introduce biases in genre
biased_genre = ["action", "action", "drama", "drama", "comedy", "comedy", "comedy", "comedy", "drama", "drama"]
df["biased_genre"] = biased_genre

# Introduce biases in year
biased_year = [2020, 2020, 2020, 2021, 2021, 2021, 2022, 2022, 2022, 2022]
df["biased_year"] = biased_year

# Add language detection to create a new column
df["detected_language"] = df["text"].apply(detect_language)

# Add additional columns for analysis
df["input_length"] = df["text"].str.len()

#Uncomment the following line
df["avg_length_ratio"] = df["input_length"] / (df["input_length"].mean())



# Step 4 - Upload the Base Dataset


In [13]:

project.upload_dataset(df, id_column="id", data_column="text", label_column="label")



/Users/shauryasingh/Documents/CMU/AI Engineering/cmu-mlip-zeno-lab/.conda/lib/python3.10/site-packages/zeno_client/util.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '2' '3' '4' '5' '6' '7' '8' '9' '10']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, id_column] = df[id_column].astype(str)
100%|██████████| 1/1 [00:00<00:00,  2.22it/s]

Successfully uploaded data


# Step 5 - Upload the AI System Outputs


In [14]:
# For demonstration, let's assume we have system predictions
df_system = pd.DataFrame(
    {
        "output": ["positive", "negative", "negative", "positive", "negative", "positive", "negative", "neutral", "positive", "negative"],
    }
)

# Create an id column to match the base dataset
df_system["id"] = df_system.index + 1

# Measure accuracy for each instance, which is averaged by the ZenoMetric above
df_system["correct"] = (df_system["output"] == df["label"]).astype(int)



# Step 6 - Upload the system outputs


In [15]:
project.upload_system(df_system, name="System A", id_column="id", output_column="output")

/Users/shauryasingh/Documents/CMU/AI Engineering/cmu-mlip-zeno-lab/.conda/lib/python3.10/site-packages/zeno_client/util.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['1' '2' '3' '4' '5' '6' '7' '8' '9' '10']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, id_column] = df[id_column].astype(str)
100%|██████████| 1/1 [00:00<00:00,  2.39it/s]

Successfully uploaded system
